# Preprocessing
**What was not removed and maybe it should be removed is the numbers!!**

**Also after stopword removel: Removal of sentences with sentence length <=5 tokens!**
We can discuss this
The other approach would be to keep the articles which have sentence length 200 (in tokens and highlight length of around 50 tokens, this will leave us with around 30.000 train data!)

In [ ]:
!pip install contractions

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import string
import nltk
#from nltk.tokenize import word_tokenize, sent_tokenize
#from nltk.stem import WordNetLemmatizer
#nltk.download('punkt')
#nltk.download('wordnet')
#from nltk.corpus import stopwords
#nltk.download('stopwords')
from string import punctuation
import contractions
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/NLG & NLU, Fall 2023, Dimitris Galanis/train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/NLG & NLU, Fall 2023, Dimitris Galanis/val.csv')
test_df = pd.read_csv('/content/drive/MyDrive/NLG & NLU, Fall 2023, Dimitris Galanis/test.csv')

In [ ]:
# For readability.
pd.set_option('max_colwidth', None)
pd.set_option('display.max_columns', None)

In [ ]:
# Hold the three datasets.
datasets = {'train': train_df, 'validation': val_df, 'test': test_df}
random_state_val = 32
for name, df in datasets.items():
    datasets[name] = df.sample(frac=0.01, random_state=random_state_val)

In [ ]:
datasets['train'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2871 entries, 15378 to 54822
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article     2871 non-null   object
 1   highlights  2871 non-null   object
 2   id          2871 non-null   object
dtypes: object(3)
memory usage: 89.7+ KB


In [ ]:
datasets['validation'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134 entries, 8477 to 8268
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article     134 non-null    object
 1   highlights  134 non-null    object
 2   id          134 non-null    object
dtypes: object(3)
memory usage: 4.2+ KB


In [ ]:
datasets['test'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115 entries, 2413 to 5353
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article     115 non-null    object
 1   highlights  115 non-null    object
 2   id          115 non-null    object
dtypes: object(3)
memory usage: 3.6+ KB


In [ ]:
# Remove Internet cluτter (potential html + href + urls + usernames).
def rm_html_url(text):
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'  # or: r'https?://\S+'
    text = re.sub(url_pattern, '', text)
    username_pattern = r'@\w+'
    text = re.sub(username_pattern, '', text)
    return text

for name, df in datasets.items():
    df['internet-free_art'] = df['article'].apply(rm_html_url)
    df['internet-free_high'] = df['highlights'].apply(rm_html_url)
    # Update the df in the 'datasets' dictionary.
    datasets[name] = df

<ipython-input-16-b5f58c8ebd29>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:
# Check the results. All OK.
#datasets['train'].head(2)
#datasets['validation'].head(2)
datasets['test'].head(2)

article  \
2413                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
from multiprocessing import Pool

# Remove article clutter (boilerplate + repetitive patterns).

# Precompile regex patterns for time + CPU efficiency:
# 1) Remove prefix '(CNN) -- ' and preceding place neames at the beginning of the text (only for Articles).
# 2) Remove 'NEW:' at the beginning (only for Highlights).
# 3) Remove parenthetical information.
# 4) Remove 'E-mail to a friend' and anything that follows.
# 5) Normalize the `\n` Highlights column in order to form nice sentences.

%%time

patterns = re.compile(r'.*\(CNN\) -- |^NEW:|\([^)]*\)|E-mail to a friend.*')
newline_pattern = re.compile(r' \.|"\n')

def rm_boiler(text):
    text = patterns.sub('', text)
    text = newline_pattern.sub('. ', text)
    return text

def apply_in_parallel(df, column):
    with Pool() as pool:
      return pool.map(rm_boiler, df[column])

# Apply function in parallel.
for name, df in datasets.items():
    df['boiler-free_art'] = apply_in_parallel(df, 'internet-free_art')
    df['boiler-free_high'] = apply_in_parallel(df, 'internet-free_high')
    datasets[name] = df

In [ ]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train'].head(2)
#datasets['validation'].head(2)
#datasets['test'].head(2)

Columns in train dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high'],
      dtype='object')
Columns in validation dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high'],
      dtype='object')
Columns in test dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high'],
      dtype='object')


,article,highlights,id,internet-free_art,internet-free_high,boiler-free_art,boiler-free_high
15378,"(CNN) -- Lukas Podolski scored his first goal since returning to Cologne but it was not enough as they crashed to a defeat 2-1 against Schalke which leaves them bottom of the German Bundesliga. Podolski celebrates his first goal since returning to Cologne, but it did not stop his side sliding to defeat. The hosts had already gone behind to a second-minute Jefferson Farfan strike when Podolski, who rejoined his boyhood club from Bayern Munich in the summer, levelled matters with six minutes played. But hopes Podolski's strike would inspire the home side fell flat a minute after half-time when Levan Kobiashvili scored a winner to move Schalke up to third in the table. Defeat for Cologne leaves them rock bottom with just one point from their opening five games. Schalke's early goal came courtesy of a corner to the far post where Gerald Asamoah was waiting to head the ball back across goal. Farfan was alert to the opportunity and sent his flying header into the back of the net for his second of the campaign. The mood inside the RheinEnergie Stadion improved significantly four minutes later when Podolski latched onto a through-ball before coolly stepping around Manuel Neuer to stroke home. The decisive moment in the game arrived just a minute after the interval when the dangerous Farfan attracted the home defence in the middle of the area before laying off a pass for Kobiashvili to thrash left-footed into the net. Meanwhile, in the day's other Bundesliga match, Werder Bremen had the better of the chances but were held to a 0-0 draw by Hannover.",Lukas Podolski scores his first goal since returning to Cologne in 2-1 defeat .\nJefferson Farfan and Levan Kobiashvili scored the goals that gave Schalke win .\nThe defeat leaves Cologne on bottom of the Bundesliga table after five games .,ac2e59e27564f0c77b03513cdea6616d8dd8361d,"(CNN) -- Lukas Podolski scored his first goal since returning to Cologne but it was not enough as they crashed to a defeat 2-1 against Schalke which leaves them bottom of the German Bundesliga. Podolski celebrates his first goal since returning to Cologne, but it did not stop his side sliding to defeat. The hosts had already gone behind to a second-minute Jefferson Farfan strike when Podolski, who rejoined his boyhood club from Bayern Munich in the summer, levelled matters with six minutes played. But hopes Podolski's strike would inspire the home side fell flat a minute after half-time when Levan Kobiashvili scored a winner to move Schalke up to third in the table. Defeat for Cologne leaves them rock bottom with just one point from their opening five games. Schalke's early goal came courtesy of a corner to the far post where Gerald Asamoah was waiting to head the ball back across goal. Farfan was alert to the opportunity and sent his flying header into the back of the net for his second of the campaign. The mood inside the RheinEnergie Stadion improved significantly four minutes later when Podolski latched onto a through-ball before coolly stepping around Manuel Neuer to stroke home. The decisive moment in the game arrived just a minute after the interval when the dangerous Farfan attracted the home defence in the middle of the area before laying off a pass for Kobiashvili to thrash left-footed into the net. Meanwhile, in the day's other Bundesliga match, Werder Bremen had the better of the chances but were held to a 0-0 draw by Hannover.",Lukas Podolski scores his first goal since returning to Cologne in 2-1 defeat .\nJefferson Farfan and Levan Kobiashvili scored the goals that gave Schalke win .\nThe defeat leaves Cologne on bottom of the Bundesliga table after five games .,"Lukas Podolski scored his first goal since returning to Cologne but it was not enough as they crashed to a defeat 2-1 against Schalke which leaves them bottom of the German Bundesliga. Podolski celebrates his first goal since returning to Co

In [ ]:
# Check for entirely non-ASCII chars.
def contains_no_ascii(text):
    return all(not char.isascii() for char in text)

for name, df in datasets.items():
    print(f"Results for {name} dataset where there are no ASCII characters:")
    print("Articles:")
    print(df['boiler-free_art'].apply(contains_no_ascii).value_counts())
    print("\nHighlights:")
    print(df['boiler-free_high'].apply(contains_no_ascii).value_counts())
    print("\n" + "-"*50 + "\n")

Results for train dataset where there are no ASCII characters:
Articles:
False    2871
Name: boiler-free_art, dtype: int64

Highlights:
False    2871
Name: boiler-free_high, dtype: int64

--------------------------------------------------

Results for validation dataset where there are no ASCII characters:
Articles:
False    134
Name: boiler-free_art, dtype: int64

Highlights:
False    134
Name: boiler-free_high, dtype: int64

--------------------------------------------------

Results for test dataset where there are no ASCII characters:
Articles:
False    115
Name: boiler-free_art, dtype: int64

Highlights:
False    115
Name: boiler-free_high, dtype: int64

--------------------------------------------------



In [ ]:
# View NLTK'S stopwords. All stopwords are lowercase.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

print(len(stop_words)) # Only 179 stopwords.
print(type(stop_words))

# Check for possessive "'s". Will it be removed together with the stopwords? NO!
if "'s" in stop_words: print('YES, possessive "s" is included.')
else: print('NO, possessive "s" is NOT included.')

179
<class 'set'>
NO, possessive "s" is NOT included.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# View SPACY'S list of stopwords.

nlp = spacy.load('en_core_web_sm')
stopwords = nlp.Defaults.stop_words
print("We will use SPACY\'s stopwords, because SPACY includes", len(stopwords), f"stopwords, which are {(len(stopwords))-(len(stop_words))} more than NLTK's stopwords. ")
print(type(stopwords))

# Check for possessive "'s". Will it be removed together with the stopwords? YES!
if "'s" in stopwords: print('YES, possessive "s" is included.')
else: print('NO, possessive "s" is NOT included.')
print('We will choose to use the SPACY library for text preprocessing and cleaning.')

We will use SPACY's stopwords, because SPACY includes 326 stopwords, which are 147 more than NLTK's stopwords. 
<class 'set'>
YES, possessive "s" is included.
We will choose to use the SPACY library for text preprocessing and cleaning.


In [ ]:
# Sentence/word tokenization and sentence processing.

def sent_cleaning(text):
    # Remove any non-ASCII characters. OK
    non_asciichars = ''.join(char for char in text if char.isascii())
    # Lowercase the text. OK
    low_text = non_asciichars.lower()
    # Expand word contractions. OK
    text = contractions.fix(low_text)
    # Sentence & word tokenization, lemmatization, and punctuation removal. OK
    doc = nlp(text)
    lemmatized_sentences = []
    for sentence in doc.sents:
        no_punct_no_stops_lemmatized_tokens = [token.lemma_ for token in sentence if not token.is_punct and not token.is_stop and token.text.strip() != ''] # filters out '\n' as a token
        lemmatized_sentences.append(no_punct_no_stops_lemmatized_tokens)
    return lemmatized_sentences

example = """These are my brother's
and sister's properties. I'm a good person. """
exx = sent_cleaning(example)
print(exx)

[['brother', 'sister', 'property'], ['good', 'person']]


In [ ]:
%%time

for name, df in datasets.items():
    df['clean_sents_art'] = df['boiler-free_art'].apply(sent_cleaning)
    df['clean_sents_high'] = df['boiler-free_high'].apply(sent_cleaning)
    datasets[name] = df

CPU times: user 7min 42s, sys: 1.08 s, total: 7min 43s
Wall time: 7min 53s


In [61]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train'].head(2)
#datasets['validation'].head(2)
#datasets['test'].head(2)

Columns in train dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high'],
      dtype='object')
Columns in validation dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high'],
      dtype='object')
Columns in test dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high'],
      dtype='object')


,article,highlights,id,internet-free_art,internet-free_high,boiler-free_art,boiler-free_high,clean_sents_art,clean_sents_high
15378,"(CNN) -- Lukas Podolski scored his first goal since returning to Cologne but it was not enough as they crashed to a defeat 2-1 against Schalke which leaves them bottom of the German Bundesliga. Podolski celebrates his first goal since returning to Cologne, but it did not stop his side sliding to defeat. The hosts had already gone behind to a second-minute Jefferson Farfan strike when Podolski, who rejoined his boyhood club from Bayern Munich in the summer, levelled matters with six minutes played. But hopes Podolski's strike would inspire the home side fell flat a minute after half-time when Levan Kobiashvili scored a winner to move Schalke up to third in the table. Defeat for Cologne leaves them rock bottom with just one point from their opening five games. Schalke's early goal came courtesy of a corner to the far post where Gerald Asamoah was waiting to head the ball back across goal. Farfan was alert to the opportunity and sent his flying header into the back of the net for his second of the campaign. The mood inside the RheinEnergie Stadion improved significantly four minutes later when Podolski latched onto a through-ball before coolly stepping around Manuel Neuer to stroke home. The decisive moment in the game arrived just a minute after the interval when the dangerous Farfan attracted the home defence in the middle of the area before laying off a pass for Kobiashvili to thrash left-footed into the net. Meanwhile, in the day's other Bundesliga match, Werder Bremen had the better of the chances but were held to a 0-0 draw by Hannover.",Lukas Podolski scores his first goal since returning to Cologne in 2-1 defeat .\nJefferson Farfan and Levan Kobiashvili scored the goals that gave Schalke win .\nThe defeat leaves Cologne on bottom of the Bundesliga table after five games .,ac2e59e27564f0c77b03513cdea6616d8dd8361d,"(CNN) -- Lukas Podolski scored his first goal since returning to Cologne but it was not enough as they crashed to a defeat 2-1 against Schalke which leaves them bottom of the German Bundesliga. Podolski celebrates his first goal since returning to Cologne, but it did not stop his side sliding to defeat. The hosts had already gone behind to a second-minute Jefferson Farfan strike when Podolski, who rejoined his boyhood club from Bayern Munich in the summer, levelled matters with six minutes played. But hopes Podolski's strike would inspire the home side fell flat a minute after half-time when Levan Kobiashvili scored a winner to move Schalke up to third in the table. Defeat for Cologne leaves them rock bottom with just one point from their opening five games. Schalke's early goal came courtesy of a corner to the far post where Gerald Asamoah was waiting to head the ball back across goal. Farfan was alert to the opportunity and sent his flying header into the back of the net for his second of the campaign. The mood inside the RheinEnergie Stadion improved significantly four minutes later when Podolski latched onto a through-ball before coolly stepping around Manuel Neuer to stroke home. The decisive moment in the game arrived just a minute after the interval when the dangerous Farfan attracted the home defence in the middle of the area before laying off a pass for Kobiashvili to thrash left-footed into the net. Meanwhile, in the day's other Bundesliga match, Werder Bremen had the better of the chances but were held to a 0-0 draw by Hannover.",Lukas Podolski scores his first goal since returning to Cologne in 2-1 defeat .\nJefferson Farfan and Levan Kobiashvili scored the goals that gave Schalke win .\nThe defeat leaves Cologne on bottom of the Bundesliga table after five games .,"Lukas Podolski scored his first goal since returning to Cologne but it was not enough as they crashed to a defeat 2-1 against Schalke which leaves them bottom of the German Bundesliga. Podolski celebrates his

In [ ]:
# check how many columns created so far. All ok.
for name, df in datasets.items():
    print(f"THe number of columns in the {name} dataset is", len(df.columns), "and these are: ", df.columns)

THe number of columns in the train dataset is 9 and these are:  Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high'],
      dtype='object')
THe number of columns in the validation dataset is 9 and these are:  Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high'],
      dtype='object')
THe number of columns in the test dataset is 9 and these are:  Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high'],
      dtype='object')


In [ ]:
for name, df in datasets.items():
  df.to_csv(f"/content/drive/MyDrive/NLG-&-NLU_Fall-2023/processed-cleaned_{name}.csv", index=False)